# import libraries 

In [1]:
from tqdm.auto import tqdm

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
import json

# Import torchvision 
import torchvision
from torchvision import datasets
from PIL import Image

from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, models
import albumentations

from sklearn.model_selection import train_test_split


import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# SET UP Device 

In [3]:

# device dog shit code 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Loading the DaTa

In [4]:
# get data loading in 

base_path = '/kaggle/input/cassava-leaf-disease-classification/'

train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'

test_path = '/kaggle/input/cassava-leaf-disease-classification/test_images'


with open(base_path+'label_num_to_disease_map.json') as f :
    mapping = json.loads(f.read())
    mapping = {int(k): v for k, v in mapping.items()}
mapping

{0: 'Cassava Bacterial Blight (CBB)',
 1: 'Cassava Brown Streak Disease (CBSD)',
 2: 'Cassava Green Mottle (CGM)',
 3: 'Cassava Mosaic Disease (CMD)',
 4: 'Healthy'}

In [5]:
train = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
train.tail()

,image_id,label
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4
21396,999998473.jpg,4


In [6]:
# Dataset responsible for manipulating data for training as well as training tests.
class DatasetLeaf(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        super().__init__()
        self.data = data.reset_index(drop=True).copy()
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data.iloc[index]
                
        image = Image.open(f'/kaggle/input/cassava-leaf-disease-classification/train_images/{item["image_id"]}')
        label = item['label']
        
        if self.transform is not None:
            image = self.transform(image)
        
        label = torch.tensor(label, dtype=torch.long)
            
        return image, label

# CONFIG 

In [7]:
BATCH_SIZE = 12
VALID_SIZE = 0.15 # percentage of data for validation .
DIM = (256, 256)
Width, Height = DIM
epochs = 12
NUM_Classes = 5
NUM_Workers = 2

# Creating Augmntations  

In [8]:

transform_train = transforms.Compose([
   # transforms.RandomRotation(0, 0.5),
    transforms.ToTensor(),
    transforms.Resize(size=DIM),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=[0.4303, 0.4967, 0.3134],
                std=[0.2330, 0.2359, 0.2237]),
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=DIM),
    transforms.Normalize(mean=[0.4303, 0.4967, 0.3134],
                std=[0.2330, 0.2359, 0.2237]),
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=DIM),
])




# Creating datasets for training and validation
train_data = DatasetLeaf(train, transform_train)
valid_data = DatasetLeaf(train, transform_valid)


# Shuffling data and choosing data that will be used for training and validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(VALID_SIZE * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=NUM_Workers, sampler=train_sampler, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, num_workers=NUM_Workers, sampler=valid_sampler, pin_memory=True)

print(f"Length train: {len(train_idx)}")
print(f"Length valid: {len(valid_idx)}")

Length train: 18188
Length valid: 3209


# Some Functions 

In [9]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               scheduler,
               lrs,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()
        lrs.append(get_lr(optimizer))
        scheduler.step()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")
    
    return train_loss.cpu(), train_acc
    

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
        return test_loss.cpu(), test_acc

# pre Trained model 

In [10]:
"""
def get_model():
    net = models.resnet152(weights=True)
    for param in net.parameters():
        param.requires_grad = True
        
    num_ft = net.fc.in_features
    net.fc = nn.Sequential(
                    nn.Linear(num_ft, 256),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(256, NUM_Classes),
                    nn.LogSoftmax(dim=1)
    )
    
    net.to(device)
    return net 
    
"""

'\ndef get_model():\n    net = models.resnet152(weights=True)\n    for param in net.parameters():\n        param.requires_grad = True\n        \n    num_ft = net.fc.in_features\n    net.fc = nn.Sequential(\n                    nn.Linear(num_ft, 256),\n                    nn.ReLU(),\n                    nn.Dropout(0.2),\n                    nn.Linear(256, NUM_Classes),\n                    nn.LogSoftmax(dim=1)\n    )\n    \n    net.to(device)\n    return net \n    \n'

In [11]:
#model = get_model()

In [12]:
# layers = list(range(8))
# i = 0
# for layer in model.children():
#     if i in layers :
#         for param in layer.parameters():
#             param.requires_grad = True
#     i += 1

In [13]:
print('hey')

hey


In [14]:
model = torch.load('/kaggle/input/resnn/pytorch/ressnt/1/model.pt')

In [15]:
 # find total parameters and trainable parameters
 total_params = sum(p.numel() for p in model.parameters())
 print(f'{total_params:,} total parameters')
 trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
 print(f'{trainable_params:,} training parameters')

58,669,637 total parameters
58,669,637 training parameters


In [16]:
"""
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
        
    f.write(request.content)
    
    
# Import accuracy metric
from helper_functions import accuracy_fn # Note: could also use torchmetrics.Accuracy(task = 'multiclass', num_classes=len(class_names)).to(device)

# Setup loss function and optimizer

loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr=0.01, steps_per_epoch=len(train_loader), epochs=5)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):

    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time
"""

'\nimport requests\nfrom pathlib import Path \n\n# Download helper functions from Learn PyTorch repo (if not already downloaded)\nif Path("helper_functions.py").is_file():\n  print("helper_functions.py already exists, skipping download")\nelse:\n  print("Downloading helper_functions.py")\n  # Note: you need the "raw" GitHub URL for this to work\n  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")\n  with open("helper_functions.py", "wb") as f:\n        \n    f.write(request.content)\n    \n    \n# Import accuracy metric\nfrom helper_functions import accuracy_fn # Note: could also use torchmetrics.Accuracy(task = \'multiclass\', num_classes=len(class_names)).to(device)\n\n# Setup loss function and optimizer\n\nloss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places\noptimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)\nscheduler = torch.optim.lr_scheduler

In [17]:
torch.cuda.empty_cache()

In [18]:
"""
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 5

 # 2. Create empty results dictionary
results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    lrs = []
    train_loss, train_acc = train_step(data_loader=train_loader, 
        model=model, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        scheduler=scheduler,
        lrs=lrs,
    
    )
    test_loss, test_acc = test_step(data_loader=valid_loader,
        model=model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )
    
    # 5. Update results dictionary
    results["train_loss"].append(train_loss.detach().numpy())
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss.detach().numpy())
    results["test_acc"].append(test_acc)


train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)
                                            
"""

'\ntorch.manual_seed(42)\n\n# Measure time\nfrom timeit import default_timer as timer\ntrain_time_start_on_gpu = timer()\n\nepochs = 5\n\n # 2. Create empty results dictionary\nresults = {"train_loss": [],\n        "train_acc": [],\n        "test_loss": [],\n        "test_acc": []\n    }\n\nfor epoch in tqdm(range(epochs)):\n    print(f"Epoch: {epoch}\n---------")\n    lrs = []\n    train_loss, train_acc = train_step(data_loader=train_loader, \n        model=model, \n        loss_fn=loss_fn,\n        optimizer=optimizer,\n        accuracy_fn=accuracy_fn,\n        scheduler=scheduler,\n        lrs=lrs,\n    \n    )\n    test_loss, test_acc = test_step(data_loader=valid_loader,\n        model=model,\n        loss_fn=loss_fn,\n        accuracy_fn=accuracy_fn\n    )\n    \n    # 5. Update results dictionary\n    results["train_loss"].append(train_loss.detach().numpy())\n    results["train_acc"].append(train_acc)\n    results["test_loss"].append(test_loss.detach().numpy())\n    results["tes

In [19]:
def plot_loss_curves(results):
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [20]:
#plot_loss_curves(results)

In [21]:
"""
results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

for i in range(1):
    test_loss, test_acc = test_step(data_loader=valid_loader,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn
        )

# 5. Update results dictionary
    results["test_loss"].append(test_loss.detach().numpy())
    results["test_acc"].append(test_acc)
    """

'\nresults = {"train_loss": [],\n        "train_acc": [],\n        "test_loss": [],\n        "test_acc": []\n    }\n\nfor i in range(1):\n    test_loss, test_acc = test_step(data_loader=valid_loader,\n            model=model,\n            loss_fn=loss_fn,\n            accuracy_fn=accuracy_fn\n        )\n\n# 5. Update results dictionary\n    results["test_loss"].append(test_loss.detach().numpy())\n    results["test_acc"].append(test_acc)\n    '

In [22]:
#torch.save(model, 'model.fpt')

In [23]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.0.conv3.weight 	 torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight 	 torch.Size([256])
layer1.0.bn3.bias 	 torch.Size([256])
layer1.0.bn3.running_mean 	 torch.Size([256])
layer1.0.bn3.running_var 	 torch.Size([256])
layer1.0.bn3.num

In [24]:
 model.parameters()

<generator object Module.parameters at 0x7e261c210510>

In [25]:
image = Image.open(f'/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg')

In [26]:
image = transform_test(image)
image = image.to(device)
image.shape

torch.Size([3, 256, 256])

In [27]:
model.eval()
with torch.inference_mode(): 
            # 1. Forward pass
            test_pred = model(image.unsqueeze(dim=0))
            test_pred = F.softmax(test_pred, dim=1)
            test_pred.argmax(axis=1)

In [28]:
sb  = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')
submission = sb.copy()
submission['label'][0] = test_pred.argmax().cpu().detach().numpy()
submission

,image_id,label
0,2216849948.jpg,4


In [29]:
submission.to_csv('submission.csv', index=False)